In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-failure-prediction/heart.csv


In [2]:
dataset = pd.read_csv('/kaggle/input/heart-failure-prediction/heart.csv')
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [4]:
features = dataset.drop('HeartDisease',axis=1)
targets = dataset['HeartDisease']

targets_onehot = pd.get_dummies(targets)
targets_onehot.head()

,0,1
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0


In [5]:
features_onehot = pd.get_dummies(features)
features_onehot.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [6]:
targets_onehot = targets_onehot.astype('float32')
features_onehot = features_onehot.astype('float32')

features_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    float32
 1   RestingBP          918 non-null    float32
 2   Cholesterol        918 non-null    float32
 3   FastingBS          918 non-null    float32
 4   MaxHR              918 non-null    float32
 5   Oldpeak            918 non-null    float32
 6   Sex_F              918 non-null    float32
 7   Sex_M              918 non-null    float32
 8   ChestPainType_ASY  918 non-null    float32
 9   ChestPainType_ATA  918 non-null    float32
 10  ChestPainType_NAP  918 non-null    float32
 11  ChestPainType_TA   918 non-null    float32
 12  RestingECG_LVH     918 non-null    float32
 13  RestingECG_Normal  918 non-null    float32
 14  RestingECG_ST      918 non-null    float32
 15  ExerciseAngina_N   918 non-null    float32
 16  ExerciseAngina_Y   918 non

In [7]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_targets, test_targets = train_test_split(features_onehot, targets_onehot, test_size=0.1, stratify=targets)
train_targets.value_counts()

0    1  
0.0  1.0    457
1.0  0.0    369
dtype: int64

In [8]:
test_targets.value_counts()

0    1  
0.0  1.0    51
1.0  0.0    41
dtype: int64

In [9]:
import tensorflow as tf

train_features_tensor = tf.convert_to_tensor(train_features)
test_features_tensor = tf.convert_to_tensor(test_features)
train_targets_tensor = tf.convert_to_tensor(train_targets)
test_targets_tensor = tf.convert_to_tensor(test_targets)

2022-11-02 09:00:18.659469: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
train_features.shape
train_targets.shape

(826, 2)

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(20, activation='sigmoid'),
    Dense(10, activation='sigmoid'),
    Dense(2, activation='sigmoid')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit(train_features_tensor, train_targets_tensor, epochs=10, batch_size=10, )

2022-11-02 09:00:19.928246: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
83/83 [==============================] - 1s 1ms/step - loss: 0.7046 - accuracy: 0.5533
Epoch 2/10
83/83 [==============================] - 0s 1ms/step - loss: 0.6530 - accuracy: 0.6429
Epoch 3/10
83/83 [==============================] - 0s 1ms/step - loss: 0.6321 - accuracy: 0.6731
Epoch 4/10
83/83 [==============================] - 0s 1ms/step - loss: 0.6041 - accuracy: 0.7155
Epoch 5/10
83/83 [==============================] - 0s 1ms/step - loss: 0.5856 - accuracy: 0.7107
Epoch 6/10
83/83 [==============================] - 0s 1ms/step - loss: 0.5764 - accuracy: 0.7300
Epoch 7/10
83/83 [==============================] - 0s 1ms/step - loss: 0.5698 - accuracy: 0.7167
Epoch 8/10
83/83 [==============================] - 0s 1ms/step - loss: 0.5538 - accuracy: 0.7288
Epoch 9/10
83/83 [==============================] - 0s 1ms/step - loss: 0.5486 - accuracy: 0.7409
Epoch 10/10
83/83 [==============================] - 0s 1ms/step - loss: 0.5390 - accuracy: 0.7409


In [13]:
model.evaluate(test_features_tensor, test_targets_tensor)

3/3 [==============================] - 0s 3ms/step - loss: 0.5355 - accuracy: 0.7065


[0.5355165600776672, 0.70652174949646]